## Topic Modeling

The process of learning, recognizing and extracting these topics across a collection of documents is called Topic Modeling

The algorithms that can be used to achieve it are LSA, pLSA, LDA, Idea2Vec

All Topic Modeling algorithms runs on following asssumptions:
1. Each document consists of a mixture of topics. 
2. Each topic consists of a collection of words.

Topic modeling models are built around the idea that the semantics of our document are actually being governed by some hidden or latent variables that we are not observing. As a result, the goal of topic modeling is to uncover these latent variables - topics - that shape meaning of document and corpus

### LSA - Latent Semantic Analysis

1. It is the foundational technique in topic modeling. 
2. The idea is to take a matrix of documents and terms and decompose it into a separate document-topic matrix and a topic-term matrix. 

First step is to generalize the document-term matrix. 
Given m documents and n words, construct mXn matrix. In which each row represents a document and each column represents a word. 

In simplest version of LSA, each entry can simply be a raw count of number of times jth word appeared in the ith document. In practice, raw counts do not perform well because they do not account for significance of each word in document. 

LSA models typically replace raw counts in document-term matrix with tf-ide score.

Tf-Idf -> Term Frequency-Inverse Document Frequency assigns a weight for term j in document:

$$ w_{i, j} = tf_{i,j} * log(N/df_{j})$$

where $$ w_{i, j} $$ is tf-idf score, $$ tf_{i,j} $$ is occurence of term in document and N is total number of docments and $$ df_{j} $$ is documents containing word. 

- A term has a large weight when it occurs frequently across document but infrequently across corpus. 
Example - The word "build" might appear often in document but because it's likely fairly common in rest of corpus, it will not have high tf-idf score . If word "genetrification" appears often in a document because it is rarer in rest of corpus, it will have a higher tf-idf score. 

Once we have document-term matrix A, we can start thinking about our latent topics. In all likelihood, A is very sparse, very noisy and very redundant across its many dimensions. As a result, to find new latent topics that can capture relationships among words and documents, we want to perform dimensionality reduction on A. 

Dimensionality reduction can be performed using truncated SVD -> Singular Value Decomposition is a technique in linear algebra that factorizes any matrix M into product of 2 separate matrices 
$$ M = U * S * V $$
s -> Diagonal matrix of singular values of M.

Critically truncated SVD reduces dimensionality by reducing selecting only t largest singular values and only keeping first t columns of U * V. In this case, t, is a hyperparameter we can select and adjust to reflect the number of topics we want to find.

$$ A = U_{t} * S_{t} * V_{t} $$

U -> Document-topic matrix
V -> Term-Topic matrix

In both U and V, columns correspond to one of the t topics.  In U rows represent document vectors expressed in terms of topics. In V, rows represent document vectors expressed in terms of topics. With these document vectors and term vector, we can now easily apply measures such as cosine similarity to evaluate:
1. The similarity of different documents. 
2. The similarity of different words. 
3. The similarity of terms (or queries) and documents (which becomes useful in information retrieval when we want to retrieve passages most relevant to our search query). 

LSA Drawbacks:
1. Lack of interpretable embeddings (we do not know what topics are and components may be arbitrarily positive or negative). 
2. Need for really large set of documents and vocabulary to get accurate results. 
3. Less efficient representations. 

### pLSA - Provbabilistic Latent Semantic Analysis

1. Uses probabilistic model instead of SVD to tackle problem. 
2. The core idea is to find a probabilistic model with latent topics that can generate data we observe in document-term matric. 

Basic Assumptions:
1. Each document consists of a mixture of topics. 
2. Each topic consists of a collection of words. 

pLSA adds a probabilistic spin to these assumptions: 
1. Given a document d, topic z is present in that document with probability P(z/d). 
2. Given a topic z, word w is drawn from z with probability P(w/z).

Joint Probability of seeing a given document and word together is:



In this case P(D), P(Z/D), P(W/Z) are parameters of our model. P(D) can be determind directly from our corpus. P(Z/D) and P(W/D) are modeled as multinomial distributions and then can be trained using expectation-minimization algorithm. EM is method of finding the likeliest parameter estimates for a model which depends on unobserved latent variables (in our case - topics). 

P(D/W) can be equivalently parameterized using a different set of 3 parameters. We can understand this equivalency by looking at the model as a generative process. In our first parameterization we were starting with the document with P(d) and then generating topic with P(z/d) and then generating word with P(w/z). In this parameterization we are starting with topic with P(z) and then independently generating topics with P(d/z) and word with P(w/z). 

pLSA really just adds a probabilistic treatment of topics and words on top of LSA. 

pLSA Drawbacks:
1. Because there are no parameters to model P(D), we don't know how to assign probabilities to new documents. 
2. The number of parameters for pLSA grows linearly with the number of documents we have so it is prone to overfitting. 

### LDA - Latent Dirichlet Allocation

1. It is a bayesian  version of pLSA. 
2. Uses Dirichlet priors for document-topic and word-topic distribution lending itself to better generalizations. 
3. Dirichlet Distribution - is actually distribution over distributions. 
4. It answers the question - given this type of distribution, what are some actual probability distributions I am likely to see. 
5. Dirichlet Distribution - provides a way of sampling probability distribution of a specific type. 

In pLSA, we sample a document then a topic based on that document and then a word based on that topic. 
LDA - From a Dirichlet Distribution Dir(alpha), we draw a random sample presenting the topic distribution or topic mixture of a particular document. This topic distribution is theta. From theta, we select a particular topic Z based on distribution. Next, from another dirichlet distribution Dir(beta), we select a random sample representing word distribution of topic Z. This word distribution is phy. From phy we choose word w. 

LDA typically works better than pLSA because it can generalize to new document easily. In pLSA, document probability is a fixed point in dataset. If we haven't seen a document, we don't have that data point. In LDA, dataset serves as training data for dirichlet distribution of document-topic distributions. If we haven't seen a document, we can easily sample from dirichlet distribution and move forward there. 

With LDA, we can extract human interpretable topics from a document corpus, where each topic is characterized by words they are most strongly associated with.

### LDA in Deep Learning - Ida2Vec

It is important to be able to extract meaning from text at every level - word, paragraph, document. At document level, we now know how to represent text as mixtures of topics. At word level, we use word2vec to obtain vector representation. 

Idea2vec is an extension of word2vec and LDA that jointly learns word, document and topic vectors. Ida2vec builds on top of skip-gram model of word2vec to generated word vectors but essentially it's a neural net that learns a word embedding by trying to use the input word to predict surrounding context words. 

With Ida2vec, instead of using word vector directly to predict context words, we leverage a context vector to make predictions. This context vector is created as sum of 2 other vectors: word vector and document vector. 

Word vector created from skip-gram. Document vector is a weighted combination of 2 other components. 
1. The document weight vector, representing weights (later to be transformed into percentages) of each topic in document. 
2. The topic matrix, representing each topic and its corresponding vector embedding.

Together the document vector and word vector generate context vectors from each word in document.

The power of Ida2vec lies in fact that it not only learns word embeddings (and context word embeddings) for words, it simultaneously learns topic representations and document representations as well.

## Implementation

## LSA - Latent Semantic Analysis

## LDA - Latent Dirichlet Allocation